## Historical Weather Forecast data in CS
<p> https://www.worldweatheronline.com/developer/api/ </p>


In [ ]:
!pip install wwo-hist
!pip install unidecode

In [72]:
from pathlib import Path
import pandas as pd
from wwo_hist import retrieve_hist_data
from typing import List, Iterable
import os
import time

In [ ]:
API_KEY   = "5ccf4c0bfb8849ddaf8204104250207"

WWO_Api_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
DONE_FILE  = WWO_Api_DIR / "Temporaria"  / "done.txt"
BASE_DIR = Path.cwd().parent.parent 
arquivo_municipios = BASE_DIR / "Municipios" / "portugalMunicipios.csv"


# MOSTRA OS CAMINHOS
print("WWO_Api_DIR:", WWO_Api_DIR)
print("DONE_FILE:", DONE_FILE)
print("arquivo_municipios:", arquivo_municipios)


WWO_Api_DIR: c:\Users\johns\Documents\Projetos\MPAlugueisLisboa\Bases\Clima\worldWeatherApi
DONE_FILE: c:\Users\johns\Documents\Projetos\MPAlugueisLisboa\Bases\Clima\worldWeatherApi\Temporaria\done.txt
arquivo_municipios: c:\Users\johns\Documents\Projetos\MPAlugueisLisboa\Bases\Municipios\portugalMunicipios.csv


In [70]:
def get_municipios_portugal(arquivo_municipios: Path = arquivo_municipios) -> List[str]:
    """
    Lê o arquivo de municípios de Portugal e retorna uma lista de cidades.
    O arquivo deve estar no formato CSV com colunas 'Regiao' e 'Cidades'.
    """

    if not arquivo_municipios.exists():
        raise FileNotFoundError(f"Arquivo de municípios não encontrado: {arquivo_municipios}")

    # ler o arquivo de municipios em um dataframe
    # separadando os campos por vírgula
    # monte uma lista concatenando: "Portugal," [Cidades]
    import pandas as pd
    df = pd.read_csv(arquivo_municipios, sep=",", encoding="utf-8")

    #ordene por Regiao e cidades
    df = df.sort_values(by=["Regiao", "Cidades"])

    # monte uma lista concatenando: "Portugal," [Cidades]
    cidades = [cidade + ",Portugal" for cidade in df["Cidades"].tolist()]

    return cidades


In [73]:

# ------------------------------------------------------------------ utilidades
def save_lines(lines: Iterable[str], file: Path) -> None:
    file.write_text("\n".join(lines), encoding="utf-8")

def read_lines(file: Path) -> List[str]:
    return file.read_text(encoding="utf-8").splitlines() if file.exists() else []

# ------------------------------------------------------------------ download
def fetch_history(city: str, start: str, end: str, freq: int = 24, pause: float = 2):
    """Baixa os dados da cidade e grava o CSV DENTRO de BASE_DIR."""
    time.sleep(pause)

    cwd = Path.cwd()            # guarda onde eu estava
    try:
        os.chdir(WWO_Api_DIR)      # muda só durante o download
        retrieve_hist_data(
            API_KEY, [city], start, end, freq,
            location_label=False, export_csv=True, store_df=True
        )
    finally:
        os.chdir(cwd)           # volta ao normal

# ------------------------------------------------------------------ orquestra
def process_weather(
    locations: List[str],
    start_date="2025-06-25",
    end_date="2025-07-01",
    freq=24,
    retries=3,
):
    done = set(read_lines(DONE_FILE))

    for city in locations:
        if city in done:
            continue

        for attempt in range(1, retries + 1):
            try:
                fetch_history(city, start_date, end_date, freq)
                done.add(city)
                save_lines(sorted(done), DONE_FILE)
                break
            except Exception as err:
                print(f"Falhou {city} ({attempt}/{retries}): {err}")
                if attempt == retries:
                    print("Desisti dessa cidade.")


In [74]:
municipios_portugal = get_municipios_portugal(arquivo_municipios)
municipios_portugal
# substua espacos por "+"
municipios_portugal = [cidade.replace(" ", "+") for cidade in municipios_portugal]
# Mostra os primeiros 10 municípios
print(municipios_portugal[:10])

['Alandroal,Portugal', 'Alcácer+do+Sal,Portugal', 'Aljustrel,Portugal', 'Almeirim,Portugal', 'Almodôvar,Portugal', 'Alpiarça,Portugal', 'Alter+do+Chão,Portugal', 'Alvito,Portugal', 'Arraiolos,Portugal', 'Arronches,Portugal']


In [56]:
process_weather(municipios_portugal)



Retrieving weather data for Alandroal,Portugal


Currently retrieving data for Alandroal,Portugal: from 2025-06-25 to 2025-06-30


C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.573859
Currently retrieving data for Alandroal,Portugal: from 2025-07-01 to 2025-07-01
Time elapsed (hh:mm:ss.ms) 0:00:01.028467


export Alandroal,Portugal completed!


Falhou Alandroal,Portugal (1/3): [Errno 2] No such file or directory: 'c:\\Users\\johns\\Documents\\Projetos\\MPAlugueisLisboa\\Bases\\Clima\\worldWeatherApi\\..\\..\\Bases\\Clima\\worldWeatherApi\\Temporaria\\done.txt'


Retrieving weather data for Alandroal,Portugal


Currently retrieving data for Alandroal,Portugal: from 2025-06-25 to 2025-06-30
Time elapsed (hh:mm:ss.ms) 0:00:00.167305
Currently retrieving data for Alandroal,Portugal: from 2025-07-01 to 2025-07-01


C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.230698


export Alandroal,Portugal completed!


Falhou Alandroal,Portugal (2/3): [Errno 2] No such file or directory: 'c:\\Users\\johns\\Documents\\Projetos\\MPAlugueisLisboa\\Bases\\Clima\\worldWeatherApi\\..\\..\\Bases\\Clima\\worldWeatherApi\\Temporaria\\done.txt'


Retrieving weather data for Alandroal,Portugal


Currently retrieving data for Alandroal,Portugal: from 2025-06-25 to 2025-06-30
Time elapsed (hh:mm:ss.ms) 0:00:00.153723
Currently retrieving data for Alandroal,Portugal: from 2025-07-01 to 2025-07-01


...

C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.265385


export Beja,Portugal completed!


Falhou Beja,Portugal (2/3): [Errno 2] No such file or directory: 'c:\\Users\\johns\\Documents\\Projetos\\MPAlugueisLisboa\\Bases\\Clima\\worldWeatherApi\\..\\..\\Bases\\Clima\\worldWeatherApi\\Temporaria\\done.txt'


Retrieving weather data for Beja,Portugal


Currently retrieving data for Beja,Portugal: from 2025-06-25 to 2025-06-30
Time elapsed (hh:mm:ss.ms) 0:00:00.155270
Currently retrieving data for Beja,Portugal: from 2025-07-01 to 2025-07-01


C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.220192


export Beja,Portugal completed!


Falhou Beja,Portugal (3/3): [Errno 2] No such file or directory: 'c:\\Users\\johns\\Documents\\Projetos\\MPAlugueisLisboa\\Bases\\Clima\\worldWeatherApi\\..\\..\\Bases\\Clima\\worldWeatherApi\\Temporaria\\done.txt'
Desisti dessa cidade.


KeyboardInterrupt: 

In [ ]:
# EXECUTA API DE CLIMA

from wwo_hist import retrieve_hist_data
import os
import unidecode
import time
import csv

api_key = "30361d43ad5f42d6a19173215230305"
base_dir = "..\\..\\Bases\\Clima\\worldWeatherApi"

class weather_py:
    def write_to_csv(self, Even_list, fileName="done.csv"):
        fileName= base_dir + "\\done.csv"
        with open(fileName, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([Even_list])

    def exportaLista(self, lst, fileName="done.csv"):
        with open(fileName, "w") as filehandle:
            for listitem in lst:
                filehandle.write("%s\n" % listitem)

    def leArquivo(self, filePath):
        lsttime = None
        with open(filePath, encoding="utf-8", errors="ignore") as f:
            lsttime = f.read().splitlines()
            lsttime = list(dict.fromkeys(lsttime))
        return lsttime

    # EXECUTA API DE CLIMA

    def process_wheather(
        self,
        location_list,
        frequency=24,
        start_date="2021-01-01",
        end_date="2021-12-30",
        retries=10,
        fileDone="..\\Temporaria\\done.csv",
        resetDoneFile=False,
    ):
        # Apaga todos arquivos para iniciar um processo novo
        if resetDoneFile and os.path.exists(fileDone):
            os.remove(fileDone)

        # cria arquivo de saida se não existir
        # esse aruivo é usado para manter o status da atualizao
        # caso pare em alguma cidade ele é consultado para voltar a partir dessa cidade
        if not os.path.exists(fileDone):
            self.write_to_csv("", fileDone)

        lst_done = []
        retry = retries
        retry_count = 0
        while retry_count <= retry:
            for location in location_list:
                lst_done = self.leArquivo(fileDone)

                if location not in lst_done:
                    try:
                        print(location)
                        time.sleep(2)
                        hist_weather_data = retrieve_hist_data(
                            api_key,
                            [location],
                            start_date,
                            end_date,
                            frequency,
                            location_label=False,
                            export_csv=True,
                            store_df=True,
                        )
                        lst_done.append(location)
                        self.exportaLista(lst_done, fileDone)
                    except:
                        self.exportaLista(lst_done, fileDone)
                        break
            retry_count = retry_count + 1
    
        
            